In [17]:
import tkinter as tk
from tkinter import messagebox
import subprocess
import random
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Constants
SENDER_EMAIL = ' '
SENDER_PASSWORD = ' '

def send_email_otp(receiver_email, otp):
    """Send OTP to the provided email address."""
    message = MIMEMultipart()
    message['From'] = SENDER_EMAIL
    message['To'] = receiver_email
    message['Subject'] = 'Your OTP Code'

    body = f"Your OTP is {otp}."
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        text = message.as_string()
        server.sendmail(SENDER_EMAIL, receiver_email, text)
        return True
    except Exception as e:
        print(f"Error sending email: {e}")
        return False
    finally:
        server.quit()

def generate_otp():
    """Generate a 6-digit OTP."""
    return str(random.randint(100000, 999999))

def open_otp_window(action):
    otp_window = tk.Toplevel(root)
    otp_window.title("OTP Verification")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    window_width = int(screen_width * 0.45)
    window_height = int(screen_height * 0.45)
    x_position = int((screen_width - window_width) / 2)
    y_position = int((screen_height - window_height) / 2)
    otp_window.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

    otp_window.configure(bg="#f0f0f0")  # Background color

    label_email = tk.Label(otp_window, text="Enter your email address:", bg="#f0f0f0", font=("Arial", 12))
    label_email.pack(pady=(20, 5))

    entry_email = tk.Entry(otp_window, width=30, font=("Arial", 12))
    entry_email.pack(pady=5)

    def send_otp():
        receiver_email = entry_email.get().strip()
        if receiver_email:
            otp = generate_otp()
            sent = send_email_otp(receiver_email, otp)
            if sent:
                otp_window.current_otp = otp
                label_status.config(text="OTP has been sent to your email.")
            else:
                messagebox.showerror("Error", "Failed to send OTP. Please try again.")
        else:
            messagebox.showerror("Error", "Please enter your email address.")

    btn_send_otp = tk.Button(otp_window, text="Send OTP", command=send_otp, font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
    btn_send_otp.pack(pady=10)

    label_enter_otp = tk.Label(otp_window, text="Enter OTP:", bg="#f0f0f0", font=("Arial", 12))
    label_enter_otp.pack(pady=10)

    entry_otp = tk.Entry(otp_window, width=10, font=("Arial", 12))
    entry_otp.pack()

    label_status = tk.Label(otp_window, text="", bg="#f0f0f0", font=("Arial", 12))
    label_status.pack(pady=10)

    def verify_otp():
        entered_otp = entry_otp.get().strip()
        if entered_otp == otp_window.current_otp:
            messagebox.showinfo("OTP Valid", "OTP is valid. Performing the action.")
            otp_window.destroy()
            if action == "disable":
                subprocess.run([r'block_usb.bat'], text=True)
                success_label.config(text="USB Ports Disabled Successfully")
            else:
                open_duration_window()
        else:
            messagebox.showerror("OTP Invalid", "Entered OTP is not valid. Please try again.")
            entry_otp.delete(0, tk.END)

    btn_verify_otp = tk.Button(otp_window, text="Verify OTP", command=verify_otp, font=("Arial", 12), bg="#008CBA", fg="white", padx=10, pady=5)
    btn_verify_otp.pack(pady=10)

def open_duration_window():
    duration_window = tk.Toplevel(root)
    duration_window.title("Enter Duration")
    duration_window.geometry("300x200")

    duration_label = tk.Label(duration_window, text="Enter duration in seconds:")
    duration_label.pack()

    duration_entry = tk.Entry(duration_window)
    duration_entry.pack()

    def set_duration():
        duration = duration_entry.get()
        if duration.isdigit():
            subprocess.run([r'unblock_usb.bat'], text=True)
            subprocess.run([r'block_usb_with_timer.bat', duration], text=True)
            duration_window.destroy()
            success_label.config(text="USB Ports Enabled Temporarily")
        else:
            messagebox.showerror("Error", "Please enter a valid number")

    duration_ok_button = tk.Button(duration_window, text="OK", command=set_duration)
    duration_ok_button.pack()

root = tk.Tk()
root.title("USB Port Security")
root.geometry("400x300")

disable_button = tk.Button(root, text="Disable USB Ports", command=lambda: open_otp_window("disable"))
disable_button.pack(pady=20)

enable_button = tk.Button(root, text="Enable USB Ports Temporarily", command=lambda: open_otp_window("enable"))
enable_button.pack(pady=20)

success_label = tk.Label(root, text="", font=("Arial", 12), bg="#f2f2f2", fg="#00ff00")
success_label.pack()

root.mainloop()